In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np

import keras
from keras import optimizers
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.resnet50 import preprocess_input

from resnet import get_resnet

In [ ]:
data_dir = '/home/ubuntu/data/'

In [ ]:
data_generator = ImageDataGenerator(
    data_format='channels_last',
    preprocessing_function=lambda x: preprocess_input(np.expand_dims(x, 0))
)

train_generator = data_generator.flow_from_directory(
    data_dir + 'train', 
    target_size=(224, 224),
    batch_size=64
)

val_generator = data_generator.flow_from_directory(
    data_dir + 'val', 
    target_size=(224, 224),
    batch_size=64
)

In [ ]:
model = get_resnet()

for layer in model.layers[:-2]:
    layer.trainable = False
    
model.layers[-2].kernel_regularizer = keras.regularizers.l2(1e-2)

In [ ]:
model.compile(
    optimizer=optimizers.Adam(lr=1e-3), 
    loss='categorical_crossentropy', metrics=['accuracy', 'top_k_categorical_accuracy']
)

In [ ]:
model.fit_generator(
    train_generator, 
    steps_per_epoch=150, epochs=10, verbose=2,
    validation_data=val_generator, validation_steps=16, 
    max_queue_size=10, workers=1, use_multiprocessing=False
)

In [ ]:
val_generator_no_shuffle = data_generator.flow_from_directory(
    data_dir + 'val', 
    target_size=(224, 224),
    batch_size=64, shuffle=False
)

In [ ]:
model.evaluate_generator(val_generator_no_shuffle, 80)

In [ ]:
model.save_weights('resnet_weights.hdf5')